In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# import getpass
# import os
# import openai


# import dotenv

# dotenv.load_dotenv()
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# openai.api_key = OPENAI_API_KEY

In [3]:
from langchain.globals import set_verbose

set_verbose(True)

In [4]:
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename
from dotenv import load_dotenv
import os
import uuid
import openai
from langchain_openai import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, CSVLoader
from langchain_community.vectorstores import Qdrant
from langchain.chains import RetrievalQA, ConversationChain
from langchain_community.llms import OpenAI
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY


In [5]:
# app = Flask(__name__)
# app.config['UPLOAD_FOLDER'] = 'uploads/'
# app.config['DATABASE_FOLDER'] = 'database/'
# ALLOWED_EXTENSIONS = {'pdf', 'csv'}

# if not os.path.exists(app.config['UPLOAD_FOLDER']):
#     os.makedirs(app.config['UPLOAD_FOLDER'])

# if not os.path.exists(app.config['DATABASE_FOLDER']):
#     os.makedirs(app.config['DATABASE_FOLDER'])



In [8]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader, CSVLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
# docs = loader.load()
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs)
# vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

ALLOWED_EXTENSIONS = {'pdf', 'csv'}

docs = []

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def process_file(filepath):
    global docs

    _, file_extension = os.path.splitext(filepath)
    file_extension = file_extension.lower()

    print(f"Processing file: {filepath}")
    
    if file_extension == '.pdf':
        loader = PyPDFLoader(filepath)
    elif file_extension == '.csv':
        loader = CSVLoader(filepath, encoding='utf-8')
    else:
        print(f"Unsupported file type: {filepath}")
        return

    docs.extend(loader.load_and_split())

def load_all_files_from_database():
    global docs
    database_folder = "database"  # 假設你的database文件夾在當前目錄
    for filename in os.listdir(database_folder):
        if allowed_file(filename):
            filepath = os.path.join(database_folder, filename)
            process_file(filepath)
            print(f"Processed file: {filename}")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(),)
    for doc in docs:
        print(doc)
    return 'All files in the database folder have been loaded and processed.', vectorstore


status, vectorstore = load_all_files_from_database()
print(status)


Processing file: database\CNS16190_correct.pdf
Processed file: CNS16190_correct.pdf
Processing file: database\ts_103701v010101p.pdf
Processed file: ts_103701v010101p.pdf
page_content='CNS16190
1C N S 1 6 1 9 0
中華⺠國國家標準  C N S  1 6 1 9 0
消費者物聯網之網宇安全：基準要求事項
C y b e r s e c u r i t y  f o r  c o n s u m e r  i n t e r n e t  o f  t h i n g s :  
B a s e l i n e  r e q u i r e m e n t s
中華⺠國 1 1 2 年 1 ⽉ 1 3 ⽇制定公布
D a t e  o f  P r o m u l g a t i o n : 2 0 2 3 1 3
⽬錄
節次
前⾔
簡介
 適⽤範圍
 引⽤標準
 ⽤語及定義、符號及縮寫
 報告實作
 消費者 IoT 裝置之網宇安全控制措施
5.1 無通⽤之預設通⾏碼
5.2 實作管理脆弱性報告之⽅式
5.3 保持軟體為最新
5.4 安全儲存敏感性安全參數
5.5 安全通訊
5.6 最⼩化暴露之攻擊⾯
5.7 確保軟體完整性
5.8 確保個⼈資料安全
5.9 使系統對中斷具韌性
5.10 檢查系統遙測資料
5.11 使⽤者容易刪除使⽤者資料' metadata={'source': 'database\\CNS16190_correct.pdf', 'page': 0}
page_content='CNS16190
25.12 使裝置容易安裝及維護
5.13 驗核輸⼊資料
 消費者 IoT 裝置之資料保護控制措施
附錄 A  參考  ) 基本概念及模型
附錄 B  參考  ) 實作符合性聲明⼀覽表
名詞對照
參考資料
( 共 3 8 ⾴  )
前⾔
本標準係依標準法之規定，經國家標準審查委員會審定，由主管機關公布之中華⺠國國家
標準。
依標準法第四條之規定，國家標準採⾃願性⽅式實施。  但經各該⽬的事業主管機關引⽤
全部或部分內容為法規者

In [17]:
vectorstore.similarity_search("""從ETSI TS 103 701中對應於控制措施(provision)5.2.1 The purpose of this test case is the conceptual assessment of the publication of the vulnerability disclosure policy. 
""", k=20)

[Document(metadata={'page': 30, 'source': 'database\\ts_103701v010101p.pdf'}, page_content='ETSI ETSI TS 1 03 701 V1.1.1 (2021 -08) 31  \nTest units \na) The TL shall  functionally check whether the vulnerability disclosure policy is publicly accessible as described \nin "Publication of Vulnerability Disclosure Policy" in IXIT 2-UserInfo. \nb) The TL shall  functionally check whether the policy contains: \n- contact information; and \n- information on timelines regarding acknowledgement of receipt and status updates. \nNOTE: Information on timelines provides flexibility to describe time values (e.g. "7 days", "quickly", etc.). \nFurther, it also allows to describe whether or how a timeline is created in the case of a reported \nvulnerability. \nAssignment of verdict \nThe verdict PASS is assigned if: \n• the vulnerability disclosure policy is publicly accessible; and \n• the vulnerability disclosure policy contains contact information and information on timelines regarding \nacknowledg

In [ ]:

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-4", request_timeout=60)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [7]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [17]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

few_shot_examples = [
    {
        "question": "根據控制措施5.1-1描述、可對抗控制措施的設備功能與TS 103 701中5.1-1Test Scenario, 生成控制措施5.1-1生成測試場景(Test Scenario)及1~2個問題供測試者檢測可對抗控制措施的設備功能是否有效\
      1.根據CNS16190 控制措施(provision): 5.1-1描述:如果使用密碼並且處於出廠預設設置以外的任何狀態，則所有消費者物聯網設備密碼應為每個設備獨有的或由用戶定義.\
      2.可對抗控制措施的設備功能(provided feature that correspond to provision description):使用者密碼透過wizard由使用者設定且與MAC Address/SN無相關性，且無法反算或暴力計算。.",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: 在CNS16190中找出控制措施(Provision)5.1-1 是甚麼?
Intermediate answer: 控制措施(Provision)5.1-1的描述是: 如果使用密碼並且處於出廠預設設置以外的任何狀態，則所有消費者物聯網設備密碼應為每個設備獨有的或由用戶定義。.
Follow up: 在 TS 103 701中對應於控制措施(provision)的測試場景(test scenario)是甚麼?
Intermediate answer: 5.1.1 Test group 5.1-1 5.1.1.0 Test group objective The test group addresses the provision 5.1-1. This test group addresses all states of the DUT with the exception of factory default. 5.1.1.1 Test case 5.1-1-1 (conceptual) Test purpose The purpose of this test case is the conceptual assessment of the password-based authentication mechanisms. Test units a) The TL shall assess for all password-based user authentication mechanisms in IXIT 1-AuthMech where passwords are not defined by the user according to "Authentication Factor" and used in any state other than the factory default whether the "Password Generation Mechanism" ensures that passwords are unique per device. Assignment of verdict The verdict PASS is assigned if: • each password of a password-based authentication mechanism being used in any state other than the factory default, that is not defined by the user, is unique per device. The verdict FAIL is assigned otherwise. 5.1.1.2 Test case 5.1-1-2 (functional) Test purpose The purpose of this test case is the functional assessment of the password-based authentication mechanisms concerning the completeness of the IXIT documentation a), the passwords defined by the user b) and the generation mechanisms c). Test units a) The TL shall functionally assess whether password-based authentication mechanisms that are not documented in IXIT 1-AuthMech are available via a network interface on the DUT or described in the user manual. EXAMPLE: Network scanning tools allow for discovery of network-based authentication mechanisms. b) For each password-based user authentication mechanism in IXIT 1-AuthMech, the TL shall functionally check whether the user is required to define all passwords that are user-defined according to "Authentication Factor" before being used. c) The TL shall functionally assess whether all passwords of the DUT, that are not defined by the user according to "Authentication Factor" in IXIT 1-AuthMech and used in any state other than the factory default, do not violate the description of the "Password Generation Mechanism". Assignment of verdict The verdict PASS is assigned if: • every discovered password-based authentication mechanism is documented in the IXIT; and • the user is required to define all passwords before being used, that are stated as defined by the user in the IXIT; and • there is no indication that the generation of a not user-defined password of the DUT used in any state other than the factory default differs from the generation mechanism described in the IXIT. The verdict FAIL is assigned otherwise.\
Follow up: 請生成有關 控制措施Provsion: 5.1-1 的測試情境(Test Scenario)。
So the final answer is: 問題：當使用密碼且處於出廠預設值（例如「admin」）以外的任何狀態時，所有消費者物聯網裝置密碼應符合下列其中一項：1. 每個設備都是唯一的。2. 由使用者定義。請詳細說明密碼是如何建立的。\

""",
    },
]
# ,
#     {
#         "question": "根據控制措施5.1-3描述、可對抗控制措施的設備功能與TS 103 701中5.1-3Test Scenario, 生成控制措施5.1-3生成測試場景(Test Scenario)及1~2個問題供測試者檢測可對抗控制措施的設備功能是否有效\
#       1.根據CNS16190 控制措施(provision) 5.1-3描述:用以對裝置鑑別使用者之鑑別機制，應使用對技術、風險及使用的性質適切之最佳實務密碼學。.\
#       2.可對抗控制措施的設備功能(provided feature that correspond to provision description):使用者密碼透過wizard由使用者設定且與MAC Address/SN無相關性，且無法反算或暴力計算。.",
#         "answer": """
# Are follow up questions needed here: Yes.
# Follow up: 在CNS16190中找出控制措施(Provision)5.1-1 是甚麼?
# Intermediate answer: 控制措施(Provision)5.1-1的描述是: 本機驗證透過 HMAC-SHA256 質詢使用使用者設定的密碼完成。 透過 HTTPs TLS v1.2 或更新的連線保護通道，chiper 套件符合 FIPS 14--3雲端管理通訊也透過 TLS v1.2 或更新的連線進行保護，chiper 套件符合 FIPS 14--3。 設備與雲端之間透過設備證書進行身份驗證。。.
# Follow up: 在 TS 103 701中對應於控制措施(provision)的測試場景(test scenario)是甚麼?
# Intermediate answer: 5.1.1 Test group 5.1-1 5.1.1.0 Test group objective The test group addresses the provision 5.1-1. This test group addresses all states of the DUT with the exception of factory default. 5.1.1.1 Test case 5.1-1-1 (conceptual) Test purpose The purpose of this test case is the conceptual assessment of the password-based authentication mechanisms. Test units a) The TL shall assess for all password-based user authentication mechanisms in IXIT 1-AuthMech where passwords are not defined by the user according to "Authentication Factor" and used in any state other than the factory default whether the "Password Generation Mechanism" ensures that passwords are unique per device. Assignment of verdict The verdict PASS is assigned if: • each password of a password-based authentication mechanism being used in any state other than the factory default, that is not defined by the user, is unique per device. The verdict FAIL is assigned otherwise. 5.1.1.2 Test case 5.1-1-2 (functional) Test purpose The purpose of this test case is the functional assessment of the password-based authentication mechanisms concerning the completeness of the IXIT documentation a), the passwords defined by the user b) and the generation mechanisms c). Test units a) The TL shall functionally assess whether password-based authentication mechanisms that are not documented in IXIT 1-AuthMech are available via a network interface on the DUT or described in the user manual. EXAMPLE: Network scanning tools allow for discovery of network-based authentication mechanisms. b) For each password-based user authentication mechanism in IXIT 1-AuthMech, the TL shall functionally check whether the user is required to define all passwords that are user-defined according to "Authentication Factor" before being used. c) The TL shall functionally assess whether all passwords of the DUT, that are not defined by the user according to "Authentication Factor" in IXIT 1-AuthMech and used in any state other than the factory default, do not violate the description of the "Password Generation Mechanism". Assignment of verdict The verdict PASS is assigned if: • every discovered password-based authentication mechanism is documented in the IXIT; and • the user is required to define all passwords before being used, that are stated as defined by the user in the IXIT; and • there is no indication that the generation of a not user-defined password of the DUT used in any state other than the factory default differs from the generation mechanism described in the IXIT. The verdict FAIL is assigned otherwise.\
# Follow up: 請生成有關 控制措施Provsion: 5.1-1 的測試情境(Test Scenario)。
# So the final answer is: 問題：5.1.3 Test group 5.1-3 5.1.3.0 Test group objective The test group addresses the provision 5.1-3. According to ETSI TS 103 645 [1]/ETSI EN 303 645 [2] best practice cryptography is defined as cryptography that is suitable for the corresponding use case and has no indication of a feasible attack with current readily available techniques. The objective of this test group is to assess, firstly, whether the cryptographic methods provide the security guarantees that are necessary for the authentication mechanisms and, secondly, whether the cryptographic methods are not known to be vulnerable to a feasible attack.5.1.3.1 Test case 5.1-3-1 (conceptual) Test purpose The purpose of this test case is the conceptual assessment of the cryptography used for the authentication mechanisms concerning the use of best practice cryptography (a-c) and the vulnerability to a feasible attack d). Test units a) For each authentication mechanism in IXIT 1-AuthMech used to authenticate users against the DUT, the TL shall assess whether the "Security Guarantees" are appropriate for the use case of user authentication, at least integrity and authenticity are required to be fulfilled. b) For each authentication mechanism in IXIT 1-AuthMech used to authenticate users against the DUT, the TL shall assess whether the mechanism according to "Description" is appropriate to achieve the "Security Guarantees". NOTE 1: A holistic approach is required to assess the security of the mechanism. c) For each authentication mechanism in IXIT 1-AuthMech used to authenticate users against the DUT, the TL shall assess whether the "Cryptographic Details" are considered as best practice cryptography for the use case of user authentication based on a reference catalogue. If "Cryptographic Details" are not included in a reference catalogue for the corresponding use case (e.g. novel cryptography), the SO shall provide evidences, e.g. a risk analysis, to justify the cryptography is appropriate as best practice for the use case. In such case the TL shall assess whether the evidence is appropriate and reliable for the use case. NOTE 2: A use case based list of examples for best practice cryptography is given in ETSI TR 103 621 [i.7]. Moreover general reference catalogues of best practice cryptography are available, for example:  SOGIS Agreed Cryptographic Mechanisms (https://www.sogis.eu). NOTE 3: A cryptographic algorithm or primitive that is deprecated with regard to its desired security property (e.g. SHA1 for collision resistance) or that relies on a cryptographic parameter (e.g. key-size) that is not appropriate, taking into account the intended lifetime of the DUT and cryptoagility, cannot be considered as best practice cryptography. d) For each authentication mechanism in IXIT 1-AuthMech used to authenticate users against the DUT, the TL shall assess whether the "Cryptographic Details" are not known to be vulnerable to a feasible attack for the desired security property on the base of the "Security Guarantees" by reference to competent cryptanalytic reports. NOTE 4: Competent cryptanalytic reports are typically published in the scientific literature or, alternatively, are to be provided by the SO. Further, clause D.2 provides information about the expected attack potential for level basic. Assignment of verdict The verdict PASS is assigned if for all user authentication mechanisms: • the security guarantees are appropriate for the use case of user authentication; and • the mechanism is appropriate to achieve the security guarantees with respect to the use case; and • all used cryptographic details are considered as best practice for the use case; and • all used cryptographic details are not known to be vulnerable to a feasible attack for the desired security property. The verdict FAIL is assigned otherwise. 5.1.3.2 Test case 5.1-3-2 (functional) Test purpose The purpose of this test case is the functional assessment of the cryptography used for the authentication mechanisms.Test units a) For each authentication mechanism in IXIT 1-AuthMech used to authenticate users against the DUT, the TL shall functionally assess whether the described "Cryptographic Details" are used by the DUT. EXAMPLE 1: Using a protocol analyser or packet sniffer tool for network-based mechanisms. EXAMPLE 2: If a PKI certificate based authentication is used, sniffing the used certificates and comparing the properties with the described cryptography in the IXIT can be helpful to collect an indication. EXAMPLE 3: If the underlying communication protocol of the authentication mechanism enables different security modes for the communication, trying to downgrade the security mode can be helpful to collect an indication. Assignment of verdict The verdict PASS is assigned if: • there is no indication that any used cryptographic setting differs from its IXIT documentation. The verdict FAIL is assigned otherwise.\
#     範例2生成的測試情境(test scenario):與裝置所使用的身份驗證方法無關，身份驗證資料在使用者和裝置之間傳輸時應進行加密。注意：傳輸通常可以在行動應用程式和裝置之間進行。注意：密碼需要始終受到保護，加密是保護密碼的標準方法。 如果密碼未加密，則它們可以簡單地讀取（即明文）並且很容易洩漏。（範例：傳輸過程中的密碼保護通常可以透過 TLS/SSL 隧道連線或 VPN 進行）。問題：1. 當使用者向裝置進行身份驗證時，身份驗證資料在傳輸過程中如何進行加密保護？\

# """,
#     }

In [18]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


qa_system_prompt = """您是一位了解CNS16190、EN303645和ETSI TS 103 701的有用助手。\
    使用以下內容來回答最後的問題。如果您不知道答案，只需說您不知道，不要試圖編造答案。
    當您被要求生成測試場景時，應該使用以下三個要素來生成測試情境(test scenario), 並根據測試場景撰寫1~3個問題提供測試者檢查：
    1. 根據Context提及的控制措施(provision)和描述(description)在CNS16190檢索控制措施(provision)內容。\
    2. 根據Context提及的可對抗控制措施的設備功能(provided feature that correspond to provision description)。\
    3. 在ETSI TS 103 701中找到對應於控制措施(provision)的測試場景(test scenario)。

    Example:{example}\

    
    Context: {context}\
    """
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

chat_history = []

In [19]:
from langchain_core.messages import HumanMessage



question = "1. 說明控制措施(provision)5.2-1:製造商應公開漏洞公布政策。本政策至少應包括：報告問題的聯繫資訊。\
    2. 從ETSI TS 103 701中對應於控制措施(provision)5.2-1的測試場景(test scenario)"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history, "example": few_shot_examples})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

# second_question = "What are common ways of doing it?"
# ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_1["answer"])

對不起，我現在無法提供ETSI TS 103 701中對應於控制措施(provision)5.2-1的測試場景(test scenario)，因為我缺乏該標準的具體內容。我建議您直接參考ETSI TS 103 701標準以獲得更準確的信息。


In [ ]:
second_question = "我現在需要請你幫我生成生成測試場景(Test Scenario)，\
      1.控制措施(provision): 5.2-1, 描述:製造商應公開漏洞公布政策。本政策至少應包括：報告問題的聯繫資訊;以及關於以下方面的時程表的資訊：(1-1）收到初始確認;以及 (1-2）狀態更新，直到報告的問題得到解決。.\
      2.可對抗控制措施的設備功能(provided feature that correspond to provision description):通過D-Link 網站（https://support.dlink.com/ReportVulnerabilities.aspx）漏洞通報資訊的聯繫方式向 D-Link 報告安全漏洞，D-Link 會及時分析安全漏洞，如果有必要會向報告者更新安全漏洞的最新狀態。 提供網站通報漏洞郵箱，以接收用戶發現的漏洞，並通過與網站告知使用者漏洞解決進度。."
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

In [ ]:
# second_question = "我現在需要請你幫我生成生成測試場景(Test Scenario)，\
#       1.條款(provision)為:5.2-1:製造商應公開漏洞公布政策。本政策至少應包括：報告問題的聯繫資訊;以及關於以下方面的時程表的資訊：(1-1）收到初始確認;以及 (1-2）狀態更新，直到報告的問題得到解決。.\
#       2.可對抗控制措施的設備功能(provided feature that correspond to provision description):通過D-Link 網站（https://support.dlink.com/ReportVulnerabilities.aspx）漏洞通報資訊的聯繫方式向 D-Link 報告安全漏洞，D-Link 會及時分析安全漏洞，如果有必要會向報告者更新安全漏洞的最新狀態。 提供網站通報漏洞郵箱，以接收用戶發現的漏洞，並通過與網站告知使用者漏洞解決進度。."
# ai_msg_3 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

# print(ai_msg_3["answer"])